## 参考文档

1. [Doc2Vec Tutorial on the Lee Dataset](https://github.com/RaReTechnologies/gensim/blob/develop/docs/notebooks/doc2vec-lee.ipynb)
2. [Tutorial 1: Corpora and Vector Spaces](https://github.com/RaReTechnologies/gensim/blob/develop/docs/notebooks/Corpora_and_Vector_Spaces.ipynb)

## 计划

参考[关于 corpus 格式的转换（与 scipy 的转换）](https://render.githubusercontent.com/view/ipynb?commit=d4c39026837770710d336849193822d032d3ad99&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f526152652d546563686e6f6c6f676965732f67656e73696d2f643463333930323638333737373037313064333336383439313933383232643033326433616439392f646f63732f6e6f7465626f6f6b732f436f72706f72615f616e645f566563746f725f5370616365732e6970796e62&nwo=RaReTechnologies%2Fgensim&path=docs%2Fnotebooks%2FCorpora_and_Vector_Spaces.ipynb&repository_id=1349775#CorpusFormats)，再参考gensim
[关于如何建立 Doc2Vec 的教程之一](https://github.com/RaReTechnologies/gensim/blob/develop/docs/notebooks/doc2vec-lee.ipynb)，拟定计划为：

1. 读取经过预处理的文件（此前已经被 newfil.pl 处理过，并保存到合适的模型中
2. [从文本建模](https://render.githubusercontent.com/view/ipynb?commit=d4c39026837770710d336849193822d032d3ad99&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f526152652d546563686e6f6c6f676965732f67656e73696d2f643463333930323638333737373037313064333336383439313933383232643033326433616439392f646f63732f6e6f7465626f6f6b732f646f63327665632d6c65652e6970796e62&nwo=RaReTechnologies%2Fgensim&path=docs%2Fnotebooks%2Fdoc2veclee.ipynb&repository_id=1349775#Training-the-Model)
3. 根据已经建立好的模型，对训练数据和测试数据[推导](https://render.githubusercontent.com/view/ipynb?commit=d4c39026837770710d336849193822d032d3ad99&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f526152652d546563686e6f6c6f676965732f67656e73696d2f643463333930323638333737373037313064333336383439313933383232643033326433616439392f646f63732f6e6f7465626f6f6b732f646f63327665632d6c65652e6970796e62&nwo=RaReTechnologies%2Fgensim&path=docs%2Fnotebooks%2Fdoc2veclee.ipynb&repository_id=1349775#Inferring-a-Vector)出向量表示（Doc2Vec
表示）
4. 处理步骤 3. 中的向量表示，保存到 DataFrame 中，使之能被 scikit-learn 接受
5. 使用 scikit-learn 进行训练
6. 评估训练好的分类器
7. 如有余力，尝试 (Word2Vec + Mean) 或 (Word2Vec + Clustering) 的方法

In [13]:
# Step 0: Import modules

import gensim
import os
import numpy as np
import pandas as pd
import random
import pandas as pd
import numpy as np
from IPython.display import display
%matplotlib inline

print "Step 0 succeed."

Step 0 succeed.


In [2]:
text8str = ""
with open(os.path.join(os.getcwd(), 'text8'), 'r') as readf:
    text8str = readf.read()

text8 = text8str.split()#gensim.corpora.dictionary.Dictionary([text8str.split()])
print len(text8)

sentences = gensim.models.word2vec.Text8Corpus('text8')
model = gensim.models.word2vec.Word2Vec(sentences=sentences, iter=10, size=200, workers=4, min_count=1)

print "Model created. See some test:"

print model.wv.most_similar(positive=['king', 'woman'], topn=1)

17005207
Model created. See some test:
[(u'queen', 0.6307931542396545)]


In [3]:
# save model
model.save('gensimModel.word2vec')

In [32]:
# Step 1: preprocess the data
paths = {}
paths['dir.train'] = os.path.join(os.getcwd(), 'trialdata', 'train')
paths['dir.test'] = os.path.join(os.getcwd(), 'trialdata', 'test')

stopwords = ""
with open('stoplist.txt', 'r') as stoplistfile:
    stopwords = stoplistfile.read()
stopwords = stopwords.split()
with open('stoplist2.txt', 'r') as stoplistfile2:
    stopwords2 = stoplistfile2.read()
    stopwords2 = stopwords2.split('\n')
    stopwords = set(stopwords)
    stopwords = list(stopwords.union(set(stopwords)))

vocabList = model.wv.vocab.items()[:200]
    
print "Step 1 Succeed"

Step 1 Succeed


In [33]:
# Step 2: read data and save it in data['vec.train'] 和 data['vec.test']

data = {}
data['vec.train'] = {'w2v.mean':[], 'class':[]}
data['vec.test'] = {'w2v.mean':[], 'class':[]}

for tpart in ['train', 'test']:
    dirpath = paths['dir.{}'.format(tpart)]
    for (ind, cls) in enumerate(os.listdir(dirpath)):
        clspath = os.path.join(dirpath, cls)
        files = os.listdir(clspath)
        for f in files:
            fpath = os.path.join(clspath, f)
            with open(fpath, 'r') as readf:
#                 print "new file"
                tokens = [token for token in readf.read().split() if token not in stopwords]#readf.read().split()#
                # Word2Vec representation
                # begin
                vec = np.array([0.0 for i in range(200)])
                expectationVal = np.array([0.0 for i in range(200)])
                for token in tokens:
                    try:
    #                     print "token: {}".format(token)
    #                     print model[token]
                        vec = model[token]
                    except:
                        vec += expectationVal
#                 print "before division: {}".format(vec)
                vec = vec / float(len(tokens))
#                 print "after division: {}".format(vec)
                 # end
            data['vec.{}'.format(tpart)]['w2v.mean'].append(vec)
            data['vec.{}'.format(tpart)]['class'].append(cls)
#             print "\n\n"
    tmp = data['vec.{}'.format(tpart)]
    ind = (random.sample(range(len(tmp['class'])), 1))[0]
    print "sample(transformed) from {}[{}]:\n[corpus]\n {}\n[class]\n{}".format(tpart, ind, tmp['w2v.mean'][ind], tmp['class'][ind])
    print

print "Step 2 Succeed"

sample(transformed) from train[189]:
[corpus]
 [-0.01121624  0.03611999  0.02182148  0.01828712 -0.01902178  0.02254571
  0.00293599  0.00988052  0.00090193  0.00979292 -0.02017714  0.00132825
 -0.03178672 -0.00410591 -0.01203715 -0.00813015  0.0042445   0.00313989
 -0.00233092  0.00733709 -0.00130573  0.00096576  0.02339144  0.01040031
  0.01760655  0.00760976 -0.01487402 -0.00450082  0.01570076  0.01612757
 -0.0169884  -0.01341857  0.00427898 -0.00463755 -0.00164583 -0.0113923
  0.01539202 -0.02718646  0.00944253 -0.0079848  -0.00341515  0.01520911
 -0.0159728  -0.01691017 -0.0141846  -0.0088249   0.00586285  0.01749247
  0.00998081 -0.02200871  0.01966882  0.0145656  -0.02671073 -0.0022228
 -0.02066257 -0.0161467   0.01079079  0.01490817 -0.03499074 -0.00257278
  0.00177303 -0.00151099 -0.00095752  0.01371672 -0.00759163  0.00145265
  0.01747983 -0.02925581 -0.0180225   0.00214752 -0.0132174   0.00024276
 -0.00360772  0.00900726 -0.00997692 -0.00737426  0.00843137 -0.02130508
  0.01

In [34]:
# tmpa = np.array([0.0 for i in range(200)])
# tmpa += model['gatech'] 
# print tmpa
# # print model['gatech']

# initialization should be the same type as elements that will be used to add

In [35]:
# Step 3: Save in Pandas.DataFrame
#
# 将 data['matrix.train'] 与 data['matrix.test'] 转换成 Pandas.DataFrame 格式，保存到 df['train'] 和 df['test'] 中（df 为字典格式：String -> DataFrame）

df = {}
for tpart in ['train', 'test']:
    datadict = {}
    datadict['class'] = data['vec.{}'.format(tpart)]['class']
    datavec = np.array(data['vec.{}'.format(tpart)]['w2v.mean'])
    for col in range(200):
        datadict[col]= datavec[:, col]

    df[tpart] = pd.DataFrame(data=datadict)
    print "See df[{}]".format(tpart)
    display(df[tpart])
    print "\n\n\n"

print "Step 3 Succeed."

# 繁琐点：研究如何把 CSR 矩阵中的数据规整好放到 DataFrame 中，并与 Class 一一对应

See df[train]


,0,1,2,3,4,5,6,7,8,9,...,191,192,193,194,195,196,197,198,199,class
0,-0.000679,0.002335,-0.000393,0.001612,0.004755,0.006500,-0.000516,0.000913,0.002337,-0.001744,...,0.000230,0.001820,-0.003358,0.002460,0.005532,-0.003370,-0.001128,0.001861,0.004854,alt.atheism
1,-0.009890,0.013064,-0.026184,0.005871,0.013187,0.005912,0.006878,-0.014619,0.002091,-0.015742,...,0.015168,0.003340,-0.019397,0.002130,-0.015077,-0.011367,0.004077,0.015664,-0.007349,alt.atheism
2,0.002480,0.000558,-0.006366,0.001636,0.004814,0.000656,-0.002975,0.002927,0.002063,0.002962,...,0.000568,0.001754,0.001674,0.002819,-0.002766,-0.004297,0.004133,0.003935,0.005866,alt.atheism
3,-0.000682,0.000152,-0.000060,0.000202,-0.000016,0.000036,-0.000081,0.000369,0.000409,-0.000012,...,0.000542,0.000205,0.000017,0.000330,0.000146,-0.000199,0.000293,0.000266,0.000087,alt.atheism
4,0.003312,0.013471,0.011227,0.004816,0.001555,0.007115,-0.003510,-0.000468,-0.002051,0.004625,...,0.002851,-0.005349,-0.003300,-0.001890,-0.003679,-0.008336,0.004773,0.003571,0.001186,alt.atheism
5,0.014844,0.001886,-0.026568,0.024739,-0.001609,0.004003,0.007723,0.014894,0.011211,-0.003149,...,0.007944,0.002879,-0.012510,0.000479,0.005023,-0.010484,-0.019967,0.008560,0.011697,alt.atheism
6,-0.002399,0.007727,0.004668,0.003912,-0.004069,0.004823,0.000628,0.002114,0.000193,0.002095,...,-0.008442,-0.001353,-0.000028,-0.000107,-0.003370,-0.002517,-0.000689,0.000938,0.004325,alt.atheism
7,0.015323,0.012870,-0.013569,-0.006185,0.002292,0.012749,0.004549,-0.013470,0.020565,-0.003315,...,0.003239,0.003483,-0.007894,0.014453,-0.006308,0.015857,-0.003020,-0.013766,0.006835,alt.atheism
8,-0.004257,0.000946,-0.000372,0.001261,-0.000098,0.000224,-0.000507,0.002307,0.002557,-0.000075,...,0.003382,0.001280,0.000109,0.002062,0.000914,-0.001242,0.001828,0.001663,0.000546,alt.atheism
9,0.017273,0.006743,-0.007178,-0.003438,0.002025,-0.000728,-0.005440,0.018605,0.010101,0.005307,...,0.003148,0.008351,0.004402,0.010129,-0.000857,0.002227,-0.001037,0.014959,-0.006065,alt.atheism






See df[test]


,0,1,2,3,4,5,6,7,8,9,...,191,192,193,194,195,196,197,198,199,class
0,0.015323,0.012870,-0.013569,-0.006185,0.002292,0.012749,0.004549,-0.013470,0.020565,-0.003315,...,0.003239,0.003483,-0.007894,0.014453,-0.006308,0.015857,-0.003020,-0.013766,0.006835,alt.atheism
1,-0.000946,-0.011530,-0.008382,0.000242,-0.003144,-0.002758,-0.003989,-0.007668,-0.001756,-0.002982,...,-0.001679,-0.006777,-0.009779,-0.002002,0.000891,0.011889,-0.002244,0.005675,0.007003,alt.atheism
2,0.008786,0.049239,0.002487,0.004490,0.001253,0.012280,-0.002981,-0.006592,-0.006149,0.009909,...,0.003429,0.003251,-0.014267,-0.000326,-0.005851,-0.007584,0.003927,0.014246,0.000223,alt.atheism
3,0.029060,0.016592,-0.009488,-0.013624,-0.028862,0.039054,-0.037528,-0.017983,-0.006731,0.002946,...,0.046660,0.004951,0.029703,0.032942,0.006420,0.015581,-0.018518,0.001164,0.003011,alt.atheism
4,0.015454,0.012980,-0.013685,-0.006238,0.002311,0.012858,0.004587,-0.013585,0.020741,-0.003343,...,0.003267,0.003512,-0.007962,0.014577,-0.006362,0.015992,-0.003045,-0.013883,0.006893,alt.atheism
5,0.005990,0.015380,0.006689,0.005139,0.001557,0.000169,0.002926,0.003295,0.001502,-0.002217,...,-0.004241,-0.008949,-0.001596,-0.002491,0.002471,-0.006706,0.011511,-0.000806,0.004040,alt.atheism
6,-0.059745,0.032420,-0.019946,0.036457,0.005532,-0.009948,-0.039510,-0.017055,0.004265,0.012365,...,0.039390,0.020318,-0.070944,0.007061,-0.042782,0.007570,-0.004510,0.048796,0.023481,alt.atheism
7,-0.051953,0.028192,-0.017345,0.031701,0.004811,-0.008651,-0.034357,-0.014831,0.003709,0.010752,...,0.034252,0.017668,-0.061691,0.006140,-0.037201,0.006583,-0.003922,0.042432,0.020418,alt.atheism
8,-0.000320,0.001101,-0.000185,0.000760,0.002241,0.003064,-0.000243,0.000431,0.001101,-0.000822,...,0.000108,0.000858,-0.001583,0.001159,0.002608,-0.001589,-0.000532,0.000877,0.002288,alt.atheism
9,0.015493,0.048397,-0.003320,0.009020,-0.011655,0.027186,-0.008282,0.007630,-0.001395,-0.000727,...,-0.000591,-0.009290,-0.003392,-0.014517,-0.002688,-0.007944,0.001311,0.001421,0.017705,alt.atheism






Step 3 Succeed.


In [36]:
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier

X_train = df['train'].drop('class', axis=1)
y_train = df['train']['class']

clf = KNeighborsClassifier()
# clf = LinearSVC()
# clf = SGDClassifier()
# clf = GaussianNB()
# clf = DecisionTreeClassifier()
# clf =MultinomialNB()
# clf = MLPClassifier()
clf.fit(X_train, y_train)
# print "Training fishied with clf with [n_classes, n_features]: {}".format(clf.coef_)
print "Step 4 finished"

Step 4 finished


In [37]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

X_test = df['test'].drop('class', axis=1)
y_test_true = df['test']['class']

y_test_pred = clf.predict(X_test)
print accuracy_score(y_test_true, y_test_pred)
print f1_score(y_test_true, y_test_pred, average='macro')
print f1_score(y_test_true, y_test_pred, average='micro')

0.322214809873
0.319243842709
0.322214809873
